In [9]:
#Import Dependencies 
from bs4 import BeautifulSoup
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import os
import requests
import time
import pandas as pd
import numpy as np

In [10]:
################ NASA Mars News ################
# Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [15]:
# Mars news URL 
news_url = "https://redplanetscience.com/"

# Retrieve page with requests module
response = requests.get(news_url)

# Create Beatiful Soup object, parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

#Print to see if URL is being printed
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet">
   <title>
    News - Mars Exploration Program
   </title>
  </link>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <s

In [18]:
# title=soup.title.text
# print(title)
paragraphs = soup.find_all('p')
for paragraph in paragraphs:
    print(paragraph.text)
    

NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities
The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
NASA to Hold Mars 2020 Perseverance Rover Launch Briefing
With supporting text below as a natural lead-in to additional content.
With supporting text below as a natural lead-in to additional content.
With supporting text below as a natural lead-in to additional content.
With supporting text below as a natural lead-in to additional content.
With supporting text below as a natural lead-in to additional content.
With supporting text below as a natural lead-in to additional content.


In [13]:
# Retrive the latest News Title 
news_title= soup.find('div', class_= 'content_title').text
news_title

AttributeError: 'NoneType' object has no attribute 'text'

In [14]:
news_paragraph =soup.find ('div', class_='article_teaser_body').text
news_paragraph

AttributeError: 'NoneType' object has no attribute 'text'